selectors
===

https://docs.python.org/3.5/library/selectors.html#module-selectors
    
Python3.4 新增的模块，是对 select 的高级封装

In [1]:
import selectors
import socket

In [4]:

sel = selectors.DefaultSelector()


def accept(sock, mask):
    conn, addr = sock.accept()  # Should be ready
    print('accepted', conn, 'from', addr)
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

    
def read(conn, mask):
    data = conn.recv(1000)  # Should be ready
    if data:
        print('echoing', repr(data), 'to', conn)
        conn.send(data)  # Hope it won't block
    else:
        print('closing', conn)
        sel.unregister(conn)
        conn.close()

        
serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # 本地复用端口
serv.bind(('localhost', 27002))
serv.listen(100)
serv.setblocking(False)
sel.register(serv, selectors.EVENT_READ, accept)  # 注册 handler、事件和 callback 函数

cli = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
cli.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
cli.connect(('localhost', 27002))
cli.send(b'test')

# while True:
events = sel.select()
for key, mask in events:
    callback = key.data
    callback(key.fileobj, mask)
    
serv.close()
cli.close()

accepted <socket.socket fd=56, family=AddressFamily.AF_INET, type=SocketType.SOCK_STREAM, proto=0, laddr=('1.1.1.1', 27002), raddr=('1.1.1.1', 64595)> from ('1.1.1.1', 64595)
